http://www.iamzxs.com/2019/08/23/%E4%BD%BF%E7%94%A8bert%E6%A8%A1%E5%9E%8B%E7%94%9F%E6%88%90token%E7%BA%A7%E5%90%91%E9%87%8F/

In [1]:
import os
import sys
curPath = os.path.abspath(os.path.dirname("__file__"))
rootPath = os.path.split(curPath)[0]
sys.path.append(rootPath)
print(sys.path)

['D:\\GitHub\\bert', 'C:\\Users\\User\\Anaconda3\\envs\\tf2.0\\python37.zip', 'C:\\Users\\User\\Anaconda3\\envs\\tf2.0\\DLLs', 'C:\\Users\\User\\Anaconda3\\envs\\tf2.0\\lib', 'C:\\Users\\User\\Anaconda3\\envs\\tf2.0', '', 'C:\\Users\\User\\AppData\\Roaming\\Python\\Python37\\site-packages', 'C:\\Users\\User\\Anaconda3\\envs\\tf2.0\\lib\\site-packages', 'C:\\Users\\User\\Anaconda3\\envs\\tf2.0\\lib\\site-packages\\win32', 'C:\\Users\\User\\Anaconda3\\envs\\tf2.0\\lib\\site-packages\\win32\\lib', 'C:\\Users\\User\\Anaconda3\\envs\\tf2.0\\lib\\site-packages\\Pythonwin', 'C:\\Users\\User\\Anaconda3\\envs\\tf2.0\\lib\\site-packages\\IPython\\extensions', 'C:\\電子電路\\.ipython', 'D:\\GitHub']


In [2]:
#!pip install bert-for-tf2
import bert

In [3]:
import tensorflow as tf
import tokenization
from tensorflow.keras.models import Model
from transformers import BertModel, BertConfig
#import modeling
import numpy as np
import h5py

In [4]:
# 配置文件
# data_root是模型文件，可以用预训练的，也可以用在分类任务上微调过的模型
data_root = './chinese_L-12_H-768_A-12/'
bert_config_file = data_root + 'bert_config.json'
bert_config = BertConfig.from_json_file(bert_config_file)
init_checkpoint = data_root + 'bert_model.ckpt'
bert_vocab_file = data_root + 'vocab.txt'
# 经过处理的输入文件路径


In [5]:
# embedding存放路径
emb_file_dir = './data/emb.h5'

# graph
#import tensorflow.compat.v1 as tf1
#tf1.disable_v2_behavior()
tf.compat.v1.disable_eager_execution()
input_ids = tf.compat.v1.placeholder(tf.int32, shape=[None, None], name='input_ids')
input_mask = tf.compat.v1.placeholder(tf.int32, shape=[None, None], name='input_masks')
segment_ids = tf.compat.v1.placeholder(tf.int32, shape=[None, None], name='segment_ids')
BATCH_SIZE = 16
SEQ_LEN = 25

In [6]:
def batch_iter(x, batch_size, shuffle=False):
    """生成批次数据，一个batch一个batch地产生句子向量"""
    data_len = len(x)
    num_batch = int((data_len - 1) / batch_size) + 1
    if shuffle:
        indices = np.random.permutation(np.arange(data_len))
        x_shuffle = np.array(x)[indices]
    else:
        x_shuffle = x[:]
    #word_mask = [[1] * (SEQ_LEN + 2) for i in range(data_len)]
    word_mask = [[1] * (SEQ_LEN) for i in range(data_len)]
    word_segment_ids = [[0] * (SEQ_LEN) for i in range(data_len)]
    #word_segment_ids = [[0] * (SEQ_LEN+2) for i in range(data_len)]]
    for i in range(num_batch):
        start_id = i * batch_size
        end_id = min((i + 1) * batch_size, data_len)
        yield x_shuffle[start_id:end_id], word_mask[start_id:end_id], word_segment_ids[start_id:end_id]

In [7]:
def read_input(file_dir):
    # 从文件中读到所有需要转化的句子
    # 这里需要做统一长度为510
    input_list = []
    df = pd.read_csv(file_dir)
    #print(type(df_sentence))
    for i in df["sentence"]:
        input_list.append(i)
    # input_list是输入list，每一个元素是一个str，代表输入文本
    # 现在需要转化成id_list
    word_id_list = []
    for query in input_list:
        split_tokens = token.tokenize(query)
        if len(split_tokens) > SEQ_LEN:
            split_tokens = split_tokens[:SEQ_LEN]
        else:
            while len(split_tokens) < SEQ_LEN:
                split_tokens.append('[PAD]')
        # ****************************************************
        # 如果是需要用到句向量，需要用这个方法
        # 加个CLS头，加个SEP尾
        tokens = []
        #tokens.append("[CLS]")
        for i_token in split_tokens:
            tokens.append(i_token)
        #tokens.append("[SEP]")
        # ****************************************************
        word_ids = token.convert_tokens_to_ids(tokens)
        word_id_list.append(word_ids)
    return word_id_list

In [8]:
# 初始化BERT
import modeling

In [9]:
#https://github.com/google-research/bert/pull/876/files
import tensorflow as tf
model = modeling.BertModel(
    config=bert_config,
    is_training=False,
    input_ids=input_ids,
    input_mask=input_mask,
    token_type_ids=segment_ids,
    use_one_hot_embeddings=False
)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.


In [10]:
# 加载BERT模型
tvars = tf.compat.v1.trainable_variables()
(assignment, initialized_variable_names) = modeling.get_assignment_map_from_checkpoint(tvars, init_checkpoint)
tf.compat.v1.train.init_from_checkpoint(init_checkpoint, assignment)

In [11]:
# 获取最后一层和倒数第二层
encoder_last_layer = model.get_sequence_output()
encoder_last2_layer = model.all_encoder_layers[-2]

In [12]:
# 读取数据
import pandas as pd
from transformers import BertTokenizer
#token = tokenization.tokenizer(vocab_file=bert_vocab_file)
bert_vocab_file = './chinese_L-12_H-768_A-12/vocab.txt'
token = BertTokenizer(vocab_file=bert_vocab_file)
input_train_data = read_input(file_dir='train134.csv')
#input_val_data = read_input(file_dir='../data/legal_domain/val_x_c.txt')
input_test_data = read_input(file_dir='test134.csv')
with tf.compat.v1.Session() as sess:
    sess.run(tf.compat.v1.global_variables_initializer())
    save_file = h5py.File('./downstream1/input_c_emb.h5', 'w')
    emb_train = []
    train_batches = batch_iter(input_train_data, batch_size=BATCH_SIZE, shuffle=False)
    for word_id, mask, segment in train_batches:
        feed_data = {input_ids: word_id, input_mask: mask, segment_ids: segment}

        last2 = sess.run(encoder_last_layer, feed_dict=feed_data)
        # print(last2.shape)
        for sub_array in last2:
            emb_train.append(sub_array)
    # 可以保存了
    emb_train_array = np.asarray(emb_train)
    save_file.create_dataset('train', data=emb_train_array)
    
    # test
    emb_test = []
    test_batches = batch_iter(input_test_data, BATCH_SIZE, shuffle=False)
    for word_id, mask, segment in test_batches:
        feed_data = {input_ids: word_id, input_mask: mask, segment_ids: segment}
        last2 = sess.run(encoder_last_layer, feed_dict=feed_data)
        # print(last2.shape)
        for sub_array in last2:
            emb_test.append(sub_array)
    # 可以保存了
    emb_test_array = np.asarray(emb_test)
    save_file.create_dataset('test', data=emb_test_array)
    save_file.close()
    print(emb_train_array.shape)
    print(emb_test_array.shape)

    # 这边目标是接下游CNN任务，因此先写入所有token的embedding，768维
    # 写入shape直接是(N, max_seq_len + 2, 768)
    # 下游需要选用的时候，如果卷积，则去掉头尾使用，如果全连接，则直接使用头部
    # 这里直接设定max_seq_len=510，加上[cls]和[sep]，得到512
    # 写入(n, 512, 768) ndarray到文件，需要用的时候再读出来，就直接舍弃embedding层

(760, 25, 768)
(160, 25, 768)


In [14]:
model8 = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(512,return_sequences=True),
    tf.keras.layers.LSTM(128,return_sequences=False),
    tf.keras.layers.Dense(64,
                activation='sigmoid' ),
    tf.keras.layers.Dense(2,
                activation='sigmoid' )
])
model8.summary()
model.compile(
optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
loss=tf.keras.losses.sparse_binary_crossentropy,
metrics=[tf.keras.metrics.sparse_binary_accuracy])


ValueError: This model has not yet been built. Build the model first by calling `build()` or calling `fit()` with some data, or specify an `input_shape` argument in the first layer(s) for automatic build.